In [ ]:
require 'irb/completion'
require 'numo/narray'
require 'numo/gsl'
require 'numo/gnuplot'
require 'ostruct'
require 'daru'
require 'fileutils'
require 'pry'
require 'spreadsheet'

In [ ]:
IRuby::Kernel.instance.switch_backend!(:pry)

### Utilities

In [ ]:
# http://youinfinitesnake.blogspot.com/2011/02/attractive-scientific-plots-with.html
# http://www.gnuplotting.org/attractive-plots/

def plotify(filename)
  Numo.noteplot do |note|
    set terminal: 'pdfcairo font "Gill Sans,9" size 21cm, 15cm linewidth 2 rounded fontscale 0.75'
    set output: filename
    
    set style: 'line 80 lt rgb "#808080"'
    set style: 'line 81 lt 0'
    
    # Remove border on top and right.  These
    # borders are useless and make it harder
    # to see plotted lines near the border.
    # Also, put it in grey; no need for so much emphasis on a border.
    set border: '3 back linestyle 80'
    set grid: 'back linestyle 81'
    set xtics: 'nomirror'
    set ytics: 'nomirror'
    
    set key: 'bottom right'
    
    # Line styles: try to pick pleasing colors, rather
    # than strictly primary colors or hard-to-see colors
    # like gnuplot's default yellow.  Make the lines thick
    # so they're easy to see in small plots in papers.
    set style: 'line 1 lt rgb "#A00000" lw 1 pt 1'
    set style: 'line 2 lt rgb "#00A000" lw 1 pt 6'
    set style: 'line 3 lt rgb "#5060D0" lw 1 pt 2'
    set style: 'line 4 lt rgb "#F25900" lw 1 pt 9'
    
    yield note
  end
end

### Data analysis

In [ ]:
eps_t_x = Numo::DFloat.linspace(800, 2000, 13)
eps_t_y = Numo::DFloat[ 0.067, 0.081, 0.105, 
                        0.119, 0.133, 0.144, 
                        0.164, 0.179, 0.195, 
                        0.209, 0.223, 0.236, 
                        0.249 ]

eps_best_fit = Numo::GSL::Fit.linear(eps_t_x, eps_t_y)
c0_eps = eps_best_fit.c0
c1_eps = eps_best_fit.c1

c0_eps_err = Math.sqrt eps_best_fit.cov00
c1_eps_err = Math.sqrt eps_best_fit.cov11

get_eps_t = -> (t) { t * c1_eps + c0_eps }

puts "eps_t = #{c0_eps} + #{c1_eps} * T"

In [ ]:
# Original data
df = Daru::DataFrame.from_csv 'work/labs/lab-8.1/data/lamp.csv'

T0_abs = -273.15 # Absolute zero
c1_temp = 1.073 # Coeff
c0_temp = -58.0 # Intercept
SA = 0.36 * 1e-4 # # Area [m^2]

df['T_br, K'] = df['T_br, C'] - T0_abs
df['T, K'] = df['T_br, K'].map { |e| e * c1_temp + c0_temp}
df['T, C'] = df['T, K'].map { |t| t + T0_abs }

df['W'] = df['U, V'] * df['I, A']
df['ln(W)'] = df['W'].map { |w| Math::log w }
df['ln(T)'] = df['T, K'].map { |t| Math::log t }
df['eps_T'] = df['T, K'].map { |t| get_eps_t.call t }

x = df['ln(T)']
y = df['ln(W)']

# Number of points to sample
n_samples = 1000

nx = Numo::DFloat[x.to_a]
ny = Numo::DFloat[y.to_a]
best_fit = Numo::GSL::Fit.linear(nx, ny)
xrange = Numo::DFloat.linspace(nx[0], nx[-1], n_samples)
best_fit_y, best_fit_err = *Numo::GSL::Fit.linear_est(xrange, best_fit)

c0 = best_fit.c0[0]
c1 = best_fit.c1[0]

c0_err = Math.sqrt best_fit.cov00[0]
c1_err = Math.sqrt best_fit.cov11[0]

df['sigma'] = (df['W'] / (df['eps_T'] * (df['T, K'] ** 4))).map { |e| e / SA }

puts "ln(W) = (#{c0} +- #{c0_err}) + (#{c1} +- #{c1_err}) * ln(T)"

In [ ]:
df

In [ ]:
T_filter = 1700.0
dffiltered = df.filter_rows { |e|  e['T, K'] >= T_filter }


In [ ]:
nsigma = Numo::DFloat[dffiltered['sigma'].to_a]

puts "sigma_mean = #{sigma_mean = nsigma.mean}"
puts "sigma_stdev = #{sigma_stdev = Numo::GSL::Stats::sd nsigma}"

In [ ]:
BOLTZMAN = 1.38 * 10 ** (-23)
SPEEDOFLIGHT = 3.0 * 10 ** 8

plank = (2 * Math::PI ** 5 * BOLTZMAN ** 4 / (15 * SPEEDOFLIGHT ** 2 * sigma_mean)) ** (1 / 3.0)
plank_sigma = (1.0 / 3) * plank / (sigma_mean) * sigma_stdev
rel_delta_sigma = plank_sigma / plank

In [ ]:
puts "plank = #{plank} +- #{plank_sigma}"

### Plots

In [ ]:
plotify 'work/labs/lab-8.1/output/plot2.pdf' do |note|
  note.set xlabel: 'ln(T)'
  note.set ylabel: 'ln(W)'
  note.set title: 'Рис. 2 Зависимость ln(W) от ln(T)'
  note.set style: 'fill transparent solid 0.5' # partial transparency
  
  note.plot [x, y, {ls:1, with:'points', title: 'ln(W)'}],
            [xrange, best_fit_y + best_fit_err,
                      best_fit_y - best_fit_err,
                {ls:3, with: 'filledcurves', title: 'Error for best fit'}],
            [xrange, best_fit_y, {ls:3, with: 'lines', title: 'Best fit c0 + c1 * ln(W)'}]
end

In [ ]:
plotify 'work/labs/lab-8.1/output/plot1.pdf' do |note|
  note.set xlabel: 'T, K'
  note.set ylabel: 'esp_T'
  note.set title: 'Рис. 1 Зависимость eps_T от T'
  
  eps_xrange = Numo::DFloat.linspace(eps_t_x[0], eps_t_x[-1], n_samples)
  eps_yrange = eps_xrange.map { |t| get_eps_t.call t}

  note.plot [eps_t_x, eps_t_y, {ls:1, with:'points', title: 'eps_T'}],
            [eps_xrange, eps_yrange, {ls:3, with: 'lines', title: 'Best fit c0 + c1 * T'}]
end

In [ ]:
df.write_excel('work/labs/lab-8.1/output/table1.xlsx')